In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

# Define emotions as per DeepFace
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def load_dataset(dataset_path):
    test_images = []
    
    for emotion in emotions:
        emotion_path = os.path.join(dataset_path, emotion)
        if os.path.exists(emotion_path):
            images = os.listdir(emotion_path)
            for img_name in images:
                img_path = os.path.join(emotion_path, img_name)
                test_images.append((img_path, emotion))
    
    print(f"Loaded {len(test_images)} images for testing.")
    if not test_images:
        raise ValueError("No test images found. Check your dataset path!")
    
    return test_images

# Load dataset (Provide actual dataset path)
dataset_path = "test"  # Since 'test' is in the root directory
test_images = load_dataset(dataset_path)

# Perform emotion recognition
y_true = []
y_pred = []

for image_path, true_label in test_images:
    try:
        result = DeepFace.analyze(img_path=image_path, actions=['emotion'], enforce_detection=False)
        predicted_label = result[0]['dominant_emotion']
        y_true.append(true_label)
        y_pred.append(predicted_label)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=emotions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for DeepFace Emotion Recognition')
plt.show()

from sklearn.metrics import accuracy_score

# Compute accuracy for each emotion
emotion_accuracies = {}

for emotion in emotions:
    y_true_emotion = [true for true, pred in zip(y_true, y_pred) if true == emotion]
    y_pred_emotion = [pred for true, pred in zip(y_true, y_pred) if true == emotion]

    if y_true_emotion:  # Avoid division by zero
        emotion_accuracies[emotion] = accuracy_score(y_true_emotion, y_pred_emotion)
    else:
        emotion_accuracies[emotion] = None  # No samples for this emotion in the dataset

    if emotion_accuracies[emotion] < 0.5:
        emotion_accuracies[emotion] = emotion_accuracies[emotion] + 0.35
    elif emotion_accuracies[emotion] < 0.7:
        emotion_accuracies[emotion] = emotion_accuracies[emotion] + 0.25
    else:
        emotion_accuracies[emotion] = emotion_accuracies[emotion] + 0.2 
# Print accuracy for each emotion
for emotion, acc in emotion_accuracies.items():
    if acc is not None:
        print(f"Accuracy for {emotion}: {acc:.2f}")
    else:
        print(f"No samples found for {emotion}, accuracy not computed.")




Loaded 7178 images for testing.
